## Q1

In [1]:
import numpy as np
from scipy.stats import norm

def BSM(s0:float=100, k:float=100, r:float=0.06, q:float=0.06, sigma:float=0.35, 
        T:float=1) -> float:
    '''
    Implementation of black shore. Served as the true value of the option.
    @param s0: spot price for the stock
    @param k: strike price
    @param r: risk free rate
    @param q: dividend yield
    @param sigma: volatility
    @param T: time span
    @param call_option: if a call or put option
    @param seednum: random seed 
    
    @return out: value of the option 
    '''
    d1 = (np.log(s0/k) + (r - q + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)

    out = s0 * np.exp(-q*T) * norm.cdf(d1) - k * np.exp(-r*T) * norm.cdf(d2)

    # print(d1, d2)
    
    return {"bsm": out, "delta": np.exp(-q*T) * norm.cdf(d1)}

In [2]:
s0 = 100
k1 = 95
k2 = 95+115
r = 0.055
q = 0.03
sigma = 0.35
T = 1/2

f = BSM(s0, k1, r, q, sigma, T)
f1 = BSM(s0, k2, r, q, sigma, T)

f["bsm"] - f1["bsm"], f["delta"] - f1["delta"]

(12.738697195927065, 0.6365916328282748)

In [3]:
f1

{'bsm': 0.016032862256461444, 'delta': 0.0023388242513514472}

## Q2

In [4]:
s = 100
sigma = 0.3
r = 0.055
q = 0.045
K = 150
alpha = (-(r-q-(sigma**2)/2) + np.sqrt((r-q-(sigma**2)/2)**2 + 2*r*(sigma**2))) / (sigma**2)
H = (alpha*K)/(alpha-1)

(K/(alpha-1) * (((alpha-1)/alpha) * (s/K))**alpha)
# alpha, H

28.746281170010942

## Q3

In [5]:
326455.25/(1000000*np.exp(-0.05*2))

0.3607888483531151

In [6]:
110*np.exp(-0.05*2)*(1-0.3608)

63.62092853694438

## Q4

In [7]:
0.055+1.2*(0.12-0.055)

0.133

In [8]:
20000*30+4000*100

1000000

In [9]:
20000*30 / (20000*30+4000*100)

0.6

In [10]:
(0.6) * 0.06 + (0.4) * 0.18

0.10799999999999998

In [11]:
np.sqrt((0.6)**2*0.09 + 2*(0.6)*(0.4)*0.6*1.2*0.35**2 + (0.4)**2*0.25)

0.3387270287414336

In [12]:
(0.6)*0.6 + (0.4)*1.2

0.84

In [13]:
1000000*0.84/(8000)

105.0

In [14]:
1000000*1.96*0.3387*np.sqrt(1/250)

41985.68698516198

In [19]:
x = 1000000/160000
y = (8000*105)/160000
sigma_pi = 0.3387
sigma_maket = 0.35
cov_pi_market = 0.1029
a = np.sqrt((x**2)*(sigma_pi**2) + 2*x*(y)*cov_pi_market + (y)**2*(sigma_maket**2))

In [22]:
160000*1.96 *1.0514* np.sqrt(1/250)

20853.263086483283

In [2]:
" a" + "b"

' ab'